In [ ]:
import torch
import detectron2
import cv2
import numpy
import math
import matplotlib.pyplot as plt
import random
import os
import numpy as np
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.logger import setup_logger
from detectron2.structures import Boxes, BoxMode, Instances
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, DatasetMapper
from detectron2.utils.visualizer import Visualizer, ColorMode

In [7]:
#Registering test dataset
dataset_name_test = "test_set" 
annotation_file_path_test = "<your-path-here>/assets/annotations.json"  
image_directory_path_test = "<your-path-here>/assets/images" 

register_coco_instances(dataset_name_test, {}, annotation_file_path_test, image_directory_path_test)

test_metadata = MetadataCatalog.get("test_set")
test_dataset_dicts = DatasetCatalog.get("test_set")

In [9]:
# Create a new cfg for testing
test_cfg = get_cfg()

# Load the config from the trained model
test_cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")) 

# Set the path to the trained weight
test_cfg.MODEL.WEIGHTS = "<your-path-here>/detectron2/model_final.pth"

# Adjust testing-specific parameters
test_cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Confidence threshold for predictions
test_cfg.DATASETS.TEST = ("test_set",)  # Make sure this matches your registered test dataset
test_cfg.DATALOADER.NUM_WORKERS = 2 # Set this equal to the number of CPU cores on your machine, decrease or increase based on the performance and system resource usage
test_cfg.INPUT.MIN_SIZE_TEST = 512 # Min image size
test_cfg.INPUT.MAX_SIZE_TEST = 512 # Max image size
test_cfg.MODEL.DEVICE = "cuda"  # Use "cpu" if you're testing on CPU
test_cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 #Number of classes the model want to detect



In [ ]:
# Initialize the predictor with the new test configuration
predictor = DefaultPredictor(test_cfg)

# Set test metadata
test_metadata = MetadataCatalog.get("test_set")

# Create test data loader with resizing
test_loader = build_detection_test_loader(
    test_cfg, 
    "test_set", 
    mapper=DatasetMapper(test_cfg, is_train=False)
)

# Output directory for visualizations
output_visualization_dir = "<your-path-here>/test_set_predictions"
os.makedirs(output_visualization_dir, exist_ok=True)

# Iterate through test set
with torch.no_grad():  # Disable gradients to save GPU memory
    for batch in test_loader:
        for data in batch:
            img = cv2.imread(data["file_name"])
            outputs = predictor(img)

            # Visualize predictions
            v = Visualizer(
                img[:, :, ::-1], 
                metadata=test_metadata,
                instance_mode=ColorMode.SEGMENTATION
            )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

            # Save visualized image
            output_filepath = os.path.join(output_visualization_dir, os.path.basename(data["file_name"]))
            cv2.imwrite(output_filepath, out.get_image()[:, :, ::-1])

print(f"Visualizations saved to: {output_visualization_dir}")

In [ ]:
# Config and model
predictor = DefaultPredictor(test_cfg)

# Paths
input_dir = "<your-path-here>/assets/images"
output_dir = "<your-path-here>"
os.makedirs(output_dir, exist_ok=True)

output_size = 1024
padding_percentage = 0.1


# Processing loop
bone_counter = 1  # Initialize global counter for naming bones

for idx, image_file in enumerate(sorted(os.listdir(input_dir))):
    if not image_file.lower().endswith((".png", ".jpg", ".jpeg", ".tif")):
        continue

    input_path = os.path.join(input_dir, image_file)
    img = cv2.imread(input_path)
    if img is None:
        print(f"⚠️ Could not read {image_file}, skipping.")
        continue

    # Run Mask R-CNN
    outputs = predictor(img)
    if len(outputs["instances"]) == 0:
        print(f"⚠️ No bone detected in {image_file}, skipping.")
        continue

    masks = outputs["instances"].pred_masks.cpu().numpy().astype(np.uint8)

    # Process each bone separately
    for b_idx, mask in enumerate(masks):
        ys, xs = np.where(mask > 0)
        if len(xs) == 0 or len(ys) == 0:
            continue  # skip empty mask

        # Bounding box
        min_x, max_x = xs.min(), xs.max()
        min_y, max_y = ys.min(), ys.max()

        # Add padding
        pad_x = int((max_x - min_x) * padding_percentage)
        pad_y = int((max_y - min_y) * padding_percentage)
        min_x = max(0, min_x - pad_x)
        min_y = max(0, min_y - pad_y)
        max_x = min(img.shape[1], max_x + pad_x)
        max_y = min(img.shape[0], max_y + pad_y)

        # Crop bone region
        cropped_img = img[min_y:max_y, min_x:max_x]
        cropped_mask = mask[min_y:max_y, min_x:max_x]

        # Apply mask
        bone_only = np.zeros_like(cropped_img)
        bone_only[cropped_mask > 0] = cropped_img[cropped_mask > 0]

        # Prepare final black 1024×1024 canvas
        black_canvas = np.zeros((output_size, output_size, 3), dtype=np.uint8)
        ch, cw = bone_only.shape[:2]

        # Resize if needed
        if ch > output_size or cw > output_size:
            scale = min(output_size / ch, output_size / cw)
            bone_only = cv2.resize(bone_only, (int(cw * scale), int(ch * scale)), interpolation=cv2.INTER_AREA)
            ch, cw = bone_only.shape[:2]

        # Center placement
        offset_y = (output_size - ch) // 2
        offset_x = (output_size - cw) // 2
        black_canvas[offset_y:offset_y+ch, offset_x:offset_x+cw] = bone_only

        # Save each bone separately
        output_name = f"{bone_counter:04d}.png" 
        cv2.imwrite(os.path.join(output_dir, output_name), black_canvas)
        print(f"✅ Saved {output_name}")
        bone_counter += 1  # Increment counter

print("All images processed.")
